In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import Packages.ClusteringHelper as ch

In [3]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
ents_data = data[data['entities'] != ''].copy()

In [4]:
ents_data = ch.add_entities_embedding(ents_data,
                                      "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings")
ents_data_filtered = ch.filter_data(ents_data, 3)

In [5]:
documents = set(ents_data.documents)

In [6]:
from Packages.TimeEvolving import DataEvolver

evolving = DataEvolver(documents, ents_data, randomly=True, step=200)

In [7]:
iterator = evolving.__iter__()
_ = iterator.__next__()

In [ ]:
current_mentions = list(evolving.get_current_data().mentions)

from textdistance import DamerauLevenshtein, Levenshtein
import numpy as np
from sklearn.cluster import dbscan


def lev_metric(x, y):
    i, j = int(x[0]), int(y[0])  # extract indices
    if len(current_mentions[i]) < 4:
        if current_mentions[i] == current_mentions[j]:
            return 0
        else:
            return Levenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower()) + 3
    else:
        return Levenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower())


def dam_lev_metric(x, y):
    i, j = int(x[0]), int(y[0])  # extract indices
    if len(current_mentions[i]) < 4:
        if current_mentions[i] == current_mentions[j]:
            return 0
        else:
            return DamerauLevenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower()) + 3
    else:
        return DamerauLevenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower())


X = np.arange(len(current_mentions)).reshape(-1, 1)
_, clusters = dbscan(X, metric=lev_metric, eps=1, min_samples=0, n_jobs=-1)

In [ ]:
current_encodings = evolving.get_current_data()['encodings'].values
current_entities = evolving.get_current_data()['entities'].values
cee_dict = {k: {'entities': [], 'mentions': [], 'encodings': [], 'sotto_clusters': []} for k in set(clusters)}
for i, cluster in enumerate(clusters):
    cee_dict[cluster]['entities'].append(current_entities[i])
    cee_dict[cluster]['mentions'].append(current_mentions[i])
    cee_dict[cluster]['encodings'].append(current_encodings[i])

In [ ]:
from sklearn.cluster import AgglomerativeClustering
clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.045, linkage="single")
for cluster in cee_dict.keys():
    try:
        cee_dict[cluster]['sotto_clusters'].append(clusterizator.fit_predict(cee_dict[cluster]['encodings']))
    except ValueError:
        cee_dict[cluster]['sotto_clusters'].append(0)

In [ ]:
for x in cee_dict:
    print(cee_dict[x]['sotto_clusters'])